In [ ]:
import ta
import numpy as np
import pandas as pd
import random

import warnings
# Turn off all warnings
warnings.filterwarnings("ignore")

In [ ]:
class Dataset():
    def __init__(self, indicators, data, name):
        self.indicators = indicators
        self.data = data
        self.data_name = name

        for indicator in indicators.keys():
            params_list = indicators.get(indicator, [{}])
            indicator_name = params_list[0]['id'].__name__

            # Buy / Sell Strings
            buy_string = indicator + "_buy"
            sell_string = indicator + "_sell"


            if indicator_name == "RSIIndicator":
                self.rsi(params_list, buy_string, sell_string)
            elif indicator_name == "StochasticOscillator":
                pass # Example indicator / Not yet implemented
            elif indicator_name == "MFIIndicator":
                pass # Example indicator / Not yet implemented
            elif indicator_name in ["AverageTrueRange", "DonchianChannel", "SMAIndicator", "EMAIndicator"]: # etc
                pass # Example indicator / Not yet implemented
            elif indicator_name == "ADXIndicator":
                self.adx(params_list, buy_string, sell_string)
            elif indicator_name == "BollingerBands":
                self.bollinger(params_list, buy_string, sell_string)
            else:
                print(indicator_name)
                raise ValueError("Unsupported indicator")
        
        print(self.indicators.keys())
        self.save()

    def bollinger(self, params_list, buy_string, sell_string):
        if len(params_list) == 2:
            indicator_bb_s = ta.volatility.BollingerBands(close=self.data["close"], window=params_list[0]['window'], window_dev=params_list[0]['window_dev'])
            indicator_bb_l = ta.volatility.BollingerBands(close=self.data["close"], window=params_list[1]['window'], window_dev=params_list[1]['window_dev'])
            df_temp = pd.DataFrame()

            # Add Bollinger Bands features
            df_temp['mavg_s'] = indicator_bb_s.bollinger_mavg()
            df_temp['mavg_m'] = indicator_bb_l.bollinger_mavg()

            self.data[buy_string] = np.where(df_temp['mavg_m'] > df_temp['mavg_s'], 1,-1)
            self.data[sell_string] = np.where(df_temp['mavg_m'] < df_temp['mavg_s'], 1,-1)

    ## copied https://medium.com/codex/algorithmic-trading-with-average-directional-index-in-python-2b5a20ecf06a   
    def adx(self, params_list, buy_string, sell_string):
        adx_indcator = ta.trend.ADXIndicator(high=self.data["high"], low=self.data["low"], close=self.data["close"], window=params_list[0]['window'])
        temp_df = pd.DataFrame(data=[adx_indcator.adx(), adx_indcator.adx_pos(), adx_indcator.adx_neg()]).transpose()

        temp_df["adx_buy"] = -1
        temp_df["adx_sell"] = -1

        for i in range(len(self.data)):
            if(temp_df["adx"].iloc[i-1]<25 and temp_df["adx"].iloc[i]> 25 and temp_df["adx_pos"].iloc[i] > temp_df["adx_neg"].iloc[i]):
                temp_df["adx_buy"].iloc[i] = 1

            if(temp_df["adx"].iloc[i-1]<25 and temp_df["adx"].iloc[i]> 25 and temp_df["adx_pos"].iloc[i] < temp_df["adx_neg"].iloc[i]):
                temp_df["adx_sell"].iloc[i] = 1

        self.data[buy_string] = temp_df["adx_buy"]
        self.data[sell_string] = temp_df["adx_sell"]

    def rsi(self, params_list, buy_string, sell_string):
        rsi_indicator = ta.momentum.RSIIndicator(close=self.data["close"], window=params_list[0]['window'])
        temp_df = pd.DataFrame(data=[rsi_indicator.rsi()]).transpose()

        temp_df["rsi_buy"] = -1
        temp_df["rsi_sell"] = -1

        for i in range(len(self.data)):
            if(temp_df["rsi"].iloc[i] < 30):
                temp_df["rsi_buy"].iloc[i] = 1

            if(temp_df["rsi"].iloc[i] > 70):
                temp_df["rsi_sell"].iloc[i] = 1

        self.data[buy_string] = temp_df["rsi_buy"]
        self.data[sell_string] = temp_df["rsi_sell"]


    ## Add your own function for an indicator here
    #def indicator(self, params):

    def save(self):
        self.data.to_csv("data/" +self.data_name+"_processed.csv", index=False)

    def __str__(self):
        return "Dataset: size = {}".format(len(self.data))

    def __repr__(self):
        return self.__str__()

In [ ]:
class BuyIndicators():
	def __init__(self,data):
		self.indicators = [column for column in data.columns if "buy" in column]
		self.size = len(self.indicators)

	def __str__(self):
		return "BuyIndicators: size = {}, indicators = {}".format(self.size, self.indicators)

	def __repr__(self):
		return self.__str__()

class SellIndicators():
	def __init__(self,data):
		self.indicators = [column for column in data.columns if "sell" in column]
		self.size = len(self.indicators)

	def __str__(self):
		return "SellIndicators: size = {}, indicators = {}".format(self.size, self.indicators)

	def __repr__(self):
		return self.__str__()

In [ ]:
"""
Defines one instance of a genome, which is a collection of genes and a fitness score.
 - Genes are a list of weights that are applied to each indicator to dictate whether to buy or sell.
 - Fitness is a score that is calculated by the fitness function, which is the total profit of the genome on BTC.
"""
class Genome():
	def __init__(self, fitness, buy_genes, sell_genes):
		self.fitness = fitness
		self.buy_genes = buy_genes
		self.sell_genes = sell_genes

	def __str__(self):
		return "Genome: fitness = {}, buy_genes = {}, sell_genes = {}".format(self.fitness, self.buy_genes, self.sell_genes)

	def __repr__(self):
		return self.__str__()

In [ ]:
import copy
best_genome = None
class TrainGenomes():
	def __init__(self, population_size, data_name, generations):
		self.population_size = population_size
		try:
			self.data = pd.read_csv("data/" +data_name+"_processed.csv")
		except:
			print("Dataset not found, please create one!")
			quit(1)
		self.generations = generations
		self.buy_genes = BuyIndicators(self.data)
		self.sell_genes = SellIndicators(self.data)

	def train(self):
		global best_genome
		# Initialize the population
		population = self.initialize_population(self.population_size, self.buy_genes, self.sell_genes)
		# Evaluate the population
		for i in range(self.generations):
			population = self.evaluate_population(population)
			# Sort the population by fitness
			population.sort(key=lambda x: x.fitness, reverse=True)
			# # Keep track of the best genome
			self.save_population(population,f"pre_genomes.csv")
			print(f"Generation {i} best genome fitness: {population[0].fitness}, average fitness: {np.mean([x.fitness for x in population])}")
			best_genome = population[0]
			# Seperate elite from population
			# do tournament selection for next generation
			population = self.tournament_selection(population, 0.05, 2)
			# # Add the elite to the population
			# population.extend(elite)
			population.sort(key=lambda x: x.fitness, reverse=True)
			self.save_population(population,f"post_genomes.csv")
			
		
		# # Keep track of the number of generations with no improvement
		# no_improvement = 0
		# # Keep track of the number of generations
		# generations = 0
		# # Keep track of the time
		# start_time = time.time()
		# Keep track of the best fitness
	def save_population(self,population,name):
		#save list of population classes as csv
		lst = [x.__dict__ for x in population]
		pd.DataFrame(lst).to_csv(f"{name}.csv")


	def initialize_population(self, population_size, buy_genes, sell_genes):
		population = []
		for i in range(population_size):
			buy_genes = np.random.uniform(-1,1,buy_genes.size)
			sell_genes = np.random.uniform(-1,1,sell_genes.size)
			population.append(Genome(0, buy_genes, sell_genes))
		return population
	
	def evaluate_population(self, population):
		for genome in population:
			genome.fitness = self.fitness(genome)

		return population
	
	def fitness(self, genome):
		buying = True
		fiat_money = 100
		btc_money = 0
		fee = 0.02 # Each buy or sell event costs 2% of current holdings.
		
		#Create a buy and sell dataframe using the genomes weights and the indicators status
		temp_df = pd.DataFrame()
		for i,indicator in enumerate(self.buy_genes.indicators):
			try:
				temp_df["buy"] += self.data[indicator] * genome.buy_genes[i]
			except KeyError:
				temp_df["buy"] = self.data[indicator] * genome.buy_genes[i]

		for i,indicator in enumerate(self.sell_genes.indicators):
			try:
				temp_df["sell"] += self.data[indicator] * genome.sell_genes[i]
			except KeyError:
				temp_df["sell"] = self.data[indicator] * genome.sell_genes[i]
		
		for index, row in self.data.iterrows():
			if buying:
				if temp_df.iloc[index]["buy"] > 0:
					btc_money = ( fiat_money / row['close'] ) * (1 - fee)
					fiat_money = 0
					buying = False

			elif temp_df.iloc[index]["sell"] > 0:
					fiat_money = (btc_money * row['close'] ) * (1 - fee)
					btc_money = 0
					buying = True
		
		return fiat_money + (btc_money * self.data.iloc[-1]['close'])
	
	def tournament_selection(self, population, elitism, tournament_size):
		# Select elite
		elite = population[:int(elitism*len(population))]
		# Initialize an empty list for the parents
		parents = []
		for _ in range(len(population) - len(elite)):
			# Randomly select tournament_size genomes
			tournament = np.random.choice(population, size=tournament_size)
			# Select the best genome
			winner = copy.deepcopy(max(tournament, key=lambda x: x.fitness))
			winner.buy_genes += np.random.uniform(-0.05, 0.05, winner.buy_genes.size)
			winner.sell_genes += np.random.uniform(-0.05, 0.05, winner.sell_genes.size)
			# Add the winner to the parents list
			parents.append(winner)
		# Add the elite to the parents list
		parents.extend(elite)
		return parents

	def mutate_population(self, population, genome_mutation_probability, gene_mutation_probability):
		# Mutate the genomes
		for genome in population:
			if np.random.uniform() < genome_mutation_probability:
				for i in range(genome.buy_genes.size):
					if np.random.uniform() < gene_mutation_probability:
						genome.buy_genes[i] += np.random.uniform(-0.05, 0.05)
				for i in range(genome.sell_genes.size):
					if np.random.uniform() < gene_mutation_probability:
						genome.sell_genes[i] += np.random.uniform(-0.05, 0.05)
	
		return population

In [ ]:

import matplotlib.pyplot as plt
class ExamineGenome():
    def __init__(self, genome, data_name):
        self.genome = genome
        try:
            self.data = pd.read_csv("data/" +data_name+"_processed.csv")
        except:
            print("Dataset not found, please create one!")
            quit(1)
        self.buy_genes = BuyIndicators(self.data)
        self.sell_genes = SellIndicators(self.data)
        self.data["timestamp"] = pd.to_datetime(self.data['timestamp'])
    def examine(self):
        buying = True
        fiat_money = 100
        btc_money = 0
        fee = 0.02 # Each buy or sell event costs 2% of current holdings.
        #Create a buy and sell dataframe using the genomes weights and the indicators status
        temp_df = pd.DataFrame()
        buys = [[],[]]
        sells = [[],[]]
        for i,indicator in enumerate(self.buy_genes.indicators):
            try:
                temp_df["buy"] += self.data[indicator] * self.genome.buy_genes[i]
            except KeyError:
                temp_df["buy"] = self.data[indicator] * self.genome.buy_genes[i]

        for i,indicator in enumerate(self.sell_genes.indicators):
            try:
                temp_df["sell"] += self.data[indicator] * self.genome.sell_genes[i]
            except KeyError:
                temp_df["sell"] = self.data[indicator] * self.genome.sell_genes[i]

        for index, row in self.data.iterrows():
            if buying:
                if temp_df.iloc[index]["buy"] > 0:
                    buys[0].append(row['timestamp'])
                    buys[1].append(row['close'])
                    btc_money = ( fiat_money / row['close'] ) * (1 - fee)
                    fiat_money = 0
                    buying = False

            elif temp_df.iloc[index]["sell"] > 0:
                    sells[0].append(row['timestamp'])
                    sells[1].append(row['close'])
                    fiat_money = (btc_money * row['close'] ) * (1 - fee)
                    btc_money = 0
                    buying = True
        
        plt.figure(figsize=(14, 8))
        plt.title('Price and Bollinger Bands')
        plt.scatter(buys[0], buys[1], label='BUY', color='green', s=25, marker="^")
        plt.scatter(sells[0], sells[1], label='SELL', color='red', s=25, marker="v")
        plt.plot(self.data['timestamp'], self.data['close'], label='Price', alpha=0.5, color='black')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend(loc='upper left')
        plt.show()
        return fiat_money + (btc_money * self.data.iloc[-1]['close'])
#print(best_genome)
#ExamineGenome(best_genome, "BTC").examine()

In [ ]:

# Indicators with optional parameters
indicators = {
    "bollinger": [{'id': ta.volatility.BollingerBands, 'window': 14, 'window_dev': 2}, {'window': 60, 'window_dev': 2}],
    "ADX_1": [{'id': ta.trend.ADXIndicator, 'window': 10}],
    "ADX_2": [{'id': ta.trend.ADXIndicator, 'window': 30}],
    "rsi": [{'id': ta.momentum.RSIIndicator, 'window': 14}],
    
}

# Indicators with optional parameters
indicators2 = {
    "bollinger": [{'id': ta.volatility.BollingerBands, 'window': 30, 'window_dev': 2}, {'window': 90, 'window_dev': 2}],
    "ADX_1": [{'id': ta.trend.ADXIndicator, 'window': 15}],
    "ADX_2": [{'id': ta.trend.ADXIndicator, 'window': 20}]
}

Dataset(indicators, pd.read_csv("raw_data.csv"), "BTC")
TrainGenomes(500, "BTC", 10).train()
ExamineGenome(best_genome, "BTC").examine()
